In [1]:
import numpy as np
import matplotlib as plt
import pandas as pd

> So please, remain that formular of Softmax regression

> $a_i = \frac{exp(z_i)}{\sum_{j=1}^{C}exp(z_j)}, \forall = 1, 2, ..., C$

> And remain: $ z_i = w_i^Tx $

In [2]:
def softmax(Z):
    e_z = np.exp(Z)
    return e_z/ e_z.sum(axis = 0)

> Because If you compute softmax in origin, So appear status overflow. Big influence to result. I solve by substract max values of per columns of Matrix Z

In [3]:
def softmax_stable(Z):
    e_z = np.exp(Z - np.max(Z, axis=0, keepdims=True))
    return e_z/e_z.sum(axis = 0)

> About loss function, I used Cross entropy for compute loss between 2 distribution 

> $ H(p, q) = -\sum_{i=1}^{C}p_ilog(q_i)$, In p is real, q is predict

Above formula, I will apply for Sotfmax

> $ J(W; x_i, y_i) = - \sum_{j=1}^C y_{ji} log(a_{ji}) $

> Gradient descent
>
> $ W = W + \triangle J(W; x_i, y_i)  \\ W = W + \lambda x_i(y_i - a_i)^T$
> 

In [4]:
import numpy as np 

# randomly generate data 
N = 2 # number of training sample 
d = 2 # data dimension 
C = 3 # number of classes 

X = np.random.randn(d, N)
y = np.random.randint(0, 3, (N,))

In [5]:
from scipy import sparse 
def convert_labels(y, C = C):
    """
    convert 1d label to a matrix label: each column of this 
    matrix coresponding to 1 element in y. In i-th column of Y, 
    only one non-zeros element located in the y[i]-th position, 
    and = 1 ex: y = [0, 2, 1, 0], and 3 classes then return

            [[1, 0, 0, 1],
             [0, 0, 1, 0],
             [0, 1, 0, 0]]
    """
    Y = sparse.coo_matrix((np.ones_like(y), 
        (y, np.arange(len(y)))), shape = (C, len(y))).toarray()
    return Y 

Y = convert_labels(y, C)

> Check Derivative

In [6]:
# cost or loss function  
def cost(X, Y, W):
    A = softmax(W.T.dot(X))
    return -np.sum(Y*np.log(A))

W_init = np.random.randn(d, C)

def grad(X, Y, W):
    A = softmax((W.T.dot(X)))
    E = A - Y
    return X.dot(E.T)
    
def numerical_grad(X, Y, W, cost):
    eps = 1e-6
    g = np.zeros_like(W)
    for i in range(W.shape[0]):
        for j in range(W.shape[1]):
            W_p = W.copy()
            W_n = W.copy()
            W_p[i, j] += eps 
            W_n[i, j] -= eps
            g[i,j] = (cost(X, Y, W_p) - cost(X, Y, W_n))/(2*eps)
    return g 

g1 = grad(X, Y, W_init)
g2 = numerical_grad(X, Y, W_init, cost)

print(np.linalg.norm(g1 - g2))

3.561252704992042e-10


### Train

In [14]:
def  softmax_regression(X, y, w_init, eta, tol = 1e-4, max_count =10000):
    W = [w_init]
    c = w_init.shape[1]
    Y = convert_labels(y, c)
    d, N = X.shape
    count = 0
    check_w_after = 20
    while count < max_count:
        mix_id = np.random.permutation(N)
        for i in mix_id:
            xi = X[:, i].reshape(d, 1)
            yi = Y[: , i].reshape(C, 1)
            ai = softmax(np.dot(W[-1].T, xi))
            w_new = W[-1] + eta*xi.dot((yi-ai).T)
            
            count+=1
            # print(count)
            if count%check_w_after == 0 and count != 0:
                if np.linalg.norm(w_new - W[-check_w_after]) < tol:
                    return W
            W.append(w_new)
    return W
eta = .05
d = X.shape[0]
W_init = np.random.randn(d, C)
W = softmax_regression(X, y, W_init, eta)

In [19]:
def pred(w, x):
    A = softmax_stable(w.T.dot(X))
    print(A)
    return np.argmax(A, axis=0)

In [20]:
pred(W[-1], X)

[[9.95771125e-01 5.00245053e-03]
 [2.42694990e-06 9.76379980e-01]
 [4.22644777e-03 1.86175693e-02]]


array([0, 1], dtype=int64)

In [18]:
y

array([0, 1])